### 构建卷积神经网络
- 卷积网络中的输入和层与传统神经网络有些区别，需重新设计，训练模块基本一致

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### 首先读取数据
- 分别构建训练集和测试集（验证集）
- DataLoader来迭代取数据

In [2]:
# 定义超参数 
input_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3  #训练的总循环周期
batch_size = 64  #一个撮（批次）的大小，64张图片

# 这里导入数据集

# 训练集
train_dataset = datasets.MNIST(root='./data',  
                            train=True,   
                            transform=transforms.ToTensor(),  
                            download=True) 

# 测试集
test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

### 卷积网络模块构建
- 一般卷积层，relu层，池化层可以写成一个套餐
- 注意卷积最后结果还是一个特征图，需要把图转换成向量才能做分类或者回归任务

卷积网络都是按套路按步骤来的
conv卷积层 - conv卷积层 - 池化pool - relu 非线性 
这是一个组合，有很多组合，所以将这些都定义在一起Sequential
nn.Conv2d是用来处理图像的，w,h 和灰度值
nn.Conv3d 是用来处理视频的，视频多了一个维度时间轴，还是图像一帧一帧组成的
nn.Conv1d 是用来处理结构化数据，一维向量的，每5个一组取一份，每5个一组,最适合就是FC，不适合做结构化数据
<img src="./img/卷积不同d的方法处理不同数据.png">
<img src="./img/卷积和第三维度的个数和channel的个数需要一致.png">
pytorch中是channel first，第一个维度是channel的个数，预处理中需要完成
<br><br><br><br><br>
其中padding的设置大部分和kernelsize的大小相关的，我们希望做到输入的维度和输出的维度不变，那么就需要通过增加padding来保持维度不缩减
<img src="./img/padding是和kernelsize选择的.png">

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # 输入大小 (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # 灰度图   说明输入是几个channel的
                out_channels=16,            # 要得到几多少个特征图，也是卷积核的特殊
                kernel_size=5,              # 卷积核大小 越小越好 一般3*3 或者5*5
                stride=1,                   # 步长
                padding=2,                  # 如果希望卷积后大小跟原来一样，需要设置padding=(kernel_size-1)/2 if stride=1
            ),                               # 输出的特征图为 (16, 28, 28)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(kernel_size=2),    # 进行池化操作（2x2 区域）, 输出结果为： (16, 14, 14) 
            # pooling对原来的数据进行压缩，一般默认都是变成原来的一半1/2，正常的池化就是变成原来的一般，除非特殊的池化操作
        )
        # 这里设置第二个层
        self.conv2 = nn.Sequential(         # 下一个套餐的输入 (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # 输出 (32, 14, 14)
            nn.ReLU(),                      # relu层
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),                # 输出 (32, 7, 7)
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(32, 64, 5, 1, 2),     # 输出 (64, 7, 7)
            nn.ReLU(),                      
            nn.MaxPool2d(2),                 # 输出 (64, 7, 7)
        )
        
        self.out = nn.Linear(64 * 7 * 7, 10)   # 全连接层得到的结果 ， 10分类

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)           # flatten操作，结果为：(batch_size, 32 * 7 * 7) 这里的view方法和reshape方法是一样的 x.size(0) 0表示batch的维度，batch不变的，-1表示自动计算 
        # 比如【5，4】 5*4的矩阵，现在转成【2，-1】那么就是【2，10】
        # 这样就将特征展开，组成了【batch，特征数】
        output = self.out(x) # 这样就可以计算全连接操作，一个特征图不能做全连接操作
        return output

### 准确率作为评估标准

In [4]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1] 
    rights = pred.eq(labels.data.view_as(pred)).sum() 
    return rights, len(labels) 

### 训练网络模型

In [5]:
# 实例化
net = CNN() 
#损失函数
criterion = nn.CrossEntropyLoss() 
#优化器
optimizer = optim.Adam(net.parameters(), lr=0.001) #定义优化器，普通的随机梯度下降算法

#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = [] 
    
    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中的每一个批进行循环
        net.train()                             
        output = net(data) 
        loss = criterion(output, target) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        right = accuracy(output, target) 
        train_rights.append(right) 

    
        if batch_idx % 100 == 0: 
            
            net.eval() 
            val_rights = [] 
            
            for (data, target) in test_loader:
                output = net(data) 
                right = accuracy(output, target) 
                val_rights.append(right)
                
            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data, 
                100. * train_r[0].numpy() / train_r[1], 
                100. * val_r[0].numpy() / val_r[1]))

当前epoch: 0 [0/60000 (0%)]	损失: 2.300574	训练集准确率: 10.94%	测试集正确率: 11.97%
当前epoch: 0 [6400/60000 (11%)]	损失: 0.239671	训练集准确率: 79.72%	测试集正确率: 94.51%
当前epoch: 0 [12800/60000 (21%)]	损失: 0.022764	训练集准确率: 87.48%	测试集正确率: 96.07%
当前epoch: 0 [19200/60000 (32%)]	损失: 0.132681	训练集准确率: 90.42%	测试集正确率: 97.67%
当前epoch: 0 [25600/60000 (43%)]	损失: 0.062406	训练集准确率: 92.09%	测试集正确率: 98.10%
当前epoch: 0 [32000/60000 (53%)]	损失: 0.010999	训练集准确率: 93.15%	测试集正确率: 98.16%
当前epoch: 0 [38400/60000 (64%)]	损失: 0.049467	训练集准确率: 93.91%	测试集正确率: 97.93%
当前epoch: 0 [44800/60000 (75%)]	损失: 0.046437	训练集准确率: 94.47%	测试集正确率: 98.39%
当前epoch: 0 [51200/60000 (85%)]	损失: 0.095034	训练集准确率: 94.93%	测试集正确率: 98.49%
当前epoch: 0 [57600/60000 (96%)]	损失: 0.125207	训练集准确率: 95.31%	测试集正确率: 98.39%
当前epoch: 1 [0/60000 (0%)]	损失: 0.008421	训练集准确率: 100.00%	测试集正确率: 98.44%
当前epoch: 1 [6400/60000 (11%)]	损失: 0.012006	训练集准确率: 98.53%	测试集正确率: 97.98%
当前epoch: 1 [12800/60000 (21%)]	损失: 0.027024	训练集准确率: 98.48%	测试集正确率: 98.49%
当前epoch: 1 [19200/60000 (32%)]	损失: 0.053866	训练集准确

### 练习
- 再加入一层卷积，效果怎么样？
- 当前任务中为什么全连接层是32*7*7 其中每一个数字代表什么含义